# 0.0 Import Libraries

In [ ]:
pip install spectral mat73  einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import math

from einops import rearrange
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from scipy import io
import torch.utils.data
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1.0 Upload Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls '/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

Autoencodermodel_trento.pth  HSI_TeSet.mat  LiDAR_TeSet.mat  TeLabel.mat      TrLabel.mat
best_model_weights.pth	     HSI_TrSet.mat  LiDAR_TrSet.mat  trento_data.mat


In [ ]:
# path
path ='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

In [ ]:
# Loader HSI TR_map_2018
trento_data=sio.loadmat(path+'trento_data.mat')['HSI_data']
trento_data.shape

(166, 600, 63)

In [ ]:
# Loader HSI TR_map_2018
trento_data=sio.loadmat(path+'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']
trento_gt=trento_data['ground']
print('trento_hsi_data shape:', trento_hsi_data.shape)
print('trento_lidar_data shape:', trento_lidar_data.shape)
print('trento_gt shape:', trento_gt.shape)


trento_hsi_data shape: (166, 600, 63)
trento_lidar_data shape: (166, 600)
trento_gt shape: (166, 600)
trento_hsi_trset shape: (2832, 144)
trento_trlabel shape: (2832, 1)
trento_hsi_teset shape: (12197, 144)
trento_telabel shape: (12197, 1)


In [ ]:
# 2.1 Define the class information
class_info = [
    (1, "Forest", 443, 54511, 54954),
    (2, "Residential Area", 423, 268219, 268642),
    (3, "Industrail Area", 499, 19067, 19566),
    (4, "Low Plants", 376, 58906, 59282),
    (5, "Soil", 331, 17095, 17426),
    (6, "Allotment", 280, 13025, 13305),
    (7, "Commenercial Area", 298, 24526, 24824),
    (8, "Water", 170, 6502, 6672)
]

class_dict = {class_number: {"class_name": class_name, "training": training, "test": test, "samples": samples} for class_number, class_name, training, test, samples in class_info}

# for class_number, class_info in class_dict.items():
#     print(f"Class {class_number}: {class_info['class_name']}")
#     print(f"Training Samples: {class_info['training']}")
#     print(f"Test Samples: {class_info['test']}")
#     print(f"Total Samples: {class_info['samples']}")
#     print()


# print(class_dict)

# 2.0 Data Preprocessing & Dataloader Preparation

In [ ]:
# # Create a mask with all class labels
# mask = np.copy(trento_gt)

# # Set the background class to 0
# mask[mask == 0] = 0

# Define patch size and stride
patch_size = 9
stride = 1

# Create an empty list to store patches and labels
hsi_samples = []
lidar_samples = []
labels = []

# Initialize a dictionary to store class count
class_count = {i: 0 for i in class_dict.keys()}

# Function to check if all classes have the required number of samples
def all_classes_completed(class_count, class_dict):
    return all(class_count[class_num] == class_dict[class_num]["samples"] for class_num in class_dict.keys())

while not all_classes_completed(class_count, class_dict):
    # Loop through the ground truth data
    for label in class_dict.keys():
        # Get the coordinates of the ground truth pixels
        #coords = np.argwhere((trento_gt == label) & (mask > 0))
        coords = np.argwhere(trento_gt == label)

        # Shuffle the coordinates to randomize the patch extraction
        np.random.shuffle(coords)

        for coord in coords:
            i, j = coord
            # Calculate the patch indices
            i_start, i_end = i - patch_size // 2, i + patch_size // 2 + 1
            j_start, j_end = j - patch_size // 2, j + patch_size // 2 + 1

            # Check if the indices are within the bounds of the HSI data
            if i_start >= 0 and i_end <= trento_hsi_data.shape[0] and j_start >= 0 and j_end <= trento_hsi_data.shape[1]:
                # Extract the patch
                hsi_patch = trento_hsi_data[i_start:i_end, j_start:j_end, :]

                # Extract the LiDAR patch
                lidar_patch = trento_lidar_data[i_start:i_end, j_start:j_end]

                # If the class count is less than the required samples
                if class_count[label] < class_dict[label]["samples"]:
                    # Append the patch and its label to the list
                    hsi_samples.append(hsi_patch)
                    lidar_samples.append(lidar_patch)
                    labels.append(label)
                    class_count[label] += 1

                    # If all classes have the required number of samples, exit the loop
                    if all_classes_completed(class_count, class_dict):
                        break

# Convert the list of patches and labels into arrays
hsi_samples = np.array(hsi_samples)
lidar_samples = np.array(lidar_samples)
labels = np.array(labels)
print('hsi_samples shape:', hsi_samples.shape)
print('lidar_samples shape:', lidar_samples.shape)
print('labels shape:', labels.shape)

KeyboardInterrupt: ignored

### 2.1  Samples Extraction

### 2.2 Training samples extraction

In [ ]:
# Create training_samples_dict based on class_dict
training_samples_dict = {class_num: class_info["training"] for class_num, class_info in class_dict.items()}

# Assuming `hsi_samples`, `lidar_samples`, and `labels` have been previously defined
# Convert the list of patches and labels into arrays if they aren't already
hsi_samples = np.array(hsi_samples)
lidar_samples = np.array(lidar_samples)
labels = np.array(labels)

# Create lists to store training and test samples and labels
hsi_training_samples, lidar_training_samples, training_labels = [], [], []
hsi_test_samples, lidar_test_samples, test_labels = [], [], []

# Split samples into training and test sets based on the desired number of training samples
for label, train_samples in training_samples_dict.items():
    # Get indices of the current class
    class_indices = np.where(labels == label)[0]

    # Randomly shuffle the indices
    np.random.shuffle(class_indices)

    # Split the indices into training and test set indices
    train_indices = class_indices[:train_samples]
    test_indices = class_indices[train_samples:]

    # Add training samples and labels for the current class
    hsi_training_samples.extend(hsi_samples[train_indices])
    lidar_training_samples.extend(lidar_samples[train_indices])
    training_labels.extend(labels[train_indices])

    # Add test samples and labels for the current class
    hsi_test_samples.extend(hsi_samples[test_indices])
    lidar_test_samples.extend(lidar_samples[test_indices])
    test_labels.extend(labels[test_indices])

# Convert lists back to numpy arrays
hsi_training_samples = np.array(hsi_training_samples)
lidar_training_samples = np.array(lidar_training_samples)
training_labels = np.array(training_labels)

hsi_test_samples = np.array(hsi_test_samples)
lidar_test_samples = np.array(lidar_test_samples)
test_labels = np.array(test_labels)

# Print shapes to verify
print('hsi_training_samples shape:', hsi_training_samples.shape)
print('lidar_training_samples shape:', lidar_training_samples.shape)
print('training_labels shape:', training_labels.shape)

print('hsi_test_samples shape:', hsi_test_samples.shape)
print('lidar_test_samples shape:', lidar_test_samples.shape)
print('test_labels shape:', test_labels.shape)

### 2.1.1 Augmented training samples

In [ ]:
import numpy as np
from scipy.ndimage import rotate

def augment_training_data(hsi_training_data, lidar_training_data, training_labels, rotations=[45, 90, 135], flip_up_down=True, flip_left_right=True):
    augmented_hsi = []
    augmented_lidar = []
    augmented_labels = []

    for hsi, lidar, label in zip(hsi_training_data, lidar_training_data, training_labels):
        # Original data
        augmented_hsi.append(hsi)
        augmented_lidar.append(lidar)
        augmented_labels.append(label)

        # Rotations
        for angle in rotations:
            hsi_rotated = rotate(hsi, angle, axes=(0, 1), reshape=False, mode='nearest')
            lidar_rotated = rotate(lidar, angle, axes=(0, 1), reshape=False, mode='nearest')

            augmented_hsi.append(hsi_rotated)
            augmented_lidar.append(lidar_rotated)
            augmented_labels.append(label)

        # Flip up-down
        if flip_up_down:
            hsi_flipped_ud = np.flipud(hsi)
            lidar_flipped_ud = np.flipud(lidar)

            augmented_hsi.append(hsi_flipped_ud)
            augmented_lidar.append(lidar_flipped_ud)
            augmented_labels.append(label)

        # Flip left-right
        if flip_left_right:
            hsi_flipped_lr = np.fliplr(hsi)
            lidar_flipped_lr = np.fliplr(lidar)

            augmented_hsi.append(hsi_flipped_lr)
            augmented_lidar.append(lidar_flipped_lr)
            augmented_labels.append(label)

    return np.array(augmented_hsi), np.array(augmented_lidar), np.array(augmented_labels)

# Augmenting the training samples
augmented_hsi_training_samples, augmented_lidar_training_samples, augmented_training_labels = augment_training_data(hsi_training_samples, lidar_training_samples, training_labels)

# Print shapes to verify the augmented training data
print('Augmented HSI training samples shape:', augmented_hsi_training_samples.shape)
print('Augmented LiDAR training samples shape:', augmented_lidar_training_samples.shape)
print('Augmented training labels shape:', augmented_training_labels.shape)


#3.0 Model Building

### 3.1 Configuration

In [ ]:
# 3.1 Configuration
class Config:
    def __init__(self,in_channels,num_patches,kernel_size,patch_size,emb_size, dim,depth,heads,dim_head,mlp_dim,num_classes,dropout,pos_emb_size,class_emb_size,stride, ):
        self.in_channels = in_channels
        self.num_patches = num_patches
        self.kernel_size = kernel_size
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.dim = dim
        self.depth = depth
        self.heads = heads
        self.dim_head = dim_head
        self.mlp_dim = mlp_dim
        self.num_classes = num_classes
        self.dropout = dropout
        self.pos_emb_size = pos_emb_size
        self.class_emb_size = class_emb_size
        self.stride = stride


### 3.2 EmbeddingPatches

In [ ]:

class PatchEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.proj = nn.Conv2d(
            config.in_channels,
            config.emb_size,
            kernel_size=config.patch_size,
            stride=config.stride,
        )
        self.pos_embedding = nn.Parameter(torch.zeros(1, config.num_patches + 1, config.emb_size))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, config.emb_size))

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        x = torch.cat((self.cls_token.expand(B, -1, -1), x), dim=1)
        x = x + self.pos_embedding[:, :x.size(1)]
        return x



### 3.2 Optianl Adding bandoutput

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, lidar_config, hsi_config):
        super(CrossAttention, self).__init__()

        # Define module parameters
        self.dim_head = lidar_config.dim_head
        self.num_patches=hsi_config.num_patches
        self.num_heads = lidar_config.heads
        self.sqrt_dim_head = math.sqrt(self.dim_head)

        # Define linear layers for transforming Q, K, and V
        self.to_q = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_k = nn.Linear(hsi_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_v = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)

        self.to_out = nn.Linear( self.num_heads* self.dim_head,self.num_patches )  # added


    def forward(self, lidar, hsi):
        B, N_lidar, _ = lidar.size()
        _, N_hsi, _ = hsi.size()

        outputs = []
        attn_scores = []  # List to store attention scores

       # Iterate over lidar and hsi patches
        for i in range(1, N_lidar):
        #for i in range(N_lidar):

            lidar_patch = lidar[:, i].unsqueeze(1)  # Add a dimension for number of patches
            for j in range(1, N_hsi):
            #for j in range(N_hsi):

                hsi_patch = hsi[:, j].unsqueeze(1)  # Add a dimension for number of patches
                Q = self.to_q(lidar_patch)
                K = self.to_k(hsi_patch)
                V = self.to_v(lidar_patch)

                Q = Q / self.sqrt_dim_head
                attn_weights = F.softmax(Q @ K.transpose(-2, -1), dim=-1)

                attn_output = attn_weights @ V
                attn_score = self.to_out(attn_output)  # added
                outputs.append(attn_output)
                attn_scores.append(attn_score)  # Store the attention scores

         # Concatenate all the outputs
        output = torch.cat(outputs, dim=1)
        attn_scores = torch.cat(attn_scores, dim=1)  # Concatenate all the attention scores

        return output, attn_scores  # Return both output and attention scores


In [ ]:
class CrossAttentionModel(nn.Module):
    def __init__(self, hsi_config, lidar_config):
        super().__init__()
        self.hsi_patch_embedding = PatchEmbedding(hsi_config)
        self.lidar_patch_embedding = PatchEmbedding(lidar_config)
        self.cross_attention = CrossAttention(lidar_config, hsi_config)

    def forward(self, lidar_data, hsi_data):
        # Apply PatchEmbedding
        lidar_emb = self.lidar_patch_embedding(lidar_data)
        hsi_emb = self.hsi_patch_embedding(hsi_data)

        # Apply CrossAttention
        output, attn_scores = self.cross_attention(lidar_emb, hsi_emb)

        return output, attn_scores  # Return both output and attention scores


In [ ]:
#3.1.1 Parameters Setting
# Hsi configuration
hsi_config = Config(
    in_channels=25,  # Each sample covers 144 bands
    num_patches=144,  # 25*1*144 bands are grouped into 3600 groups
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=1,  # Adjusted to match new patch size (5*5, 144/24=6)
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=6,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


# Lidara configuration
lidar_config = Config(
    in_channels=25,  # lidar group has 1 channels
    num_patches=1,  # 1 band for Lidar
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=1, # Adjusted to match new patch size
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=6,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch = training_hsi_samples  # Shape: (2832, 5, 5, 144)
lidar_batch = training_lidar_samples  # Shape: (2832, 5, 5, 1)
print('hsi_batch sahpe before transpose:', hsi_batch.shape)
print('lidar_batch sahpe before transpose:', lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
hsi_batch = hsi_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 144, 5, 5)
lidar_batch = lidar_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 1, 5, 5)
print('hsi_batch sahpe after transpose:', hsi_batch.shape)
print('lidar_batch shape after transpose:', lidar_batch.shape)

hsi_batch sahpe before transpose: (2832, 5, 5, 144)
lidar_batch sahpe before transpose: (2832, 5, 5, 1)
hsi_batch sahpe after transpose: (2832, 144, 5, 5)
lidar_batch shape after transpose: (2832, 1, 5, 5)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = training_hsi_samples[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = training_lidar_samples[:1]  # Shape: (1, 5, 5, 1)
print('one_hsi_batch sahpe before transpose:', one_hsi_batch.shape)
print('one_lidar_batch sahpe before transpose:', one_lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
one_hsi_batch = one_hsi_batch.transpose(0, 3, 1, 2)  # New shape: (1, 144, 5, 5)
one_lidar_batch = one_lidar_batch.transpose(0, 3, 1, 2)  # New shape: (1, 1, 5, 5)
print('hsi_batch sahpe after transpose:', one_hsi_batch.shape)
print('lidar_batch shape after transpose:', one_lidar_batch.shape)

one_hsi_batch sahpe before transpose: (1, 5, 5, 144)
one_lidar_batch sahpe before transpose: (1, 5, 5, 1)
hsi_batch sahpe after transpose: (1, 144, 5, 5)
lidar_batch shape after transpose: (1, 1, 5, 5)


In [ ]:
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = training_hsi_samples[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = training_lidar_samples[:1]  # Shape: (1, 5, 5, 1)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
one_hsi_batch_flat = torch.from_numpy(one_hsi_batch.astype(np.float32).reshape(1, 25, 144, 1)).to(device)
one_lidar_batch_flat = torch.from_numpy(one_lidar_batch.astype(np.float32).reshape(1, 25, 1, 1)).to(device)

# Initialize the patch embedding module
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
one_hsi_batch_embedded = hsi_patch_embedding(one_hsi_batch_flat).to(device)
one_lidar_batch_embedded = lidar_patch_embedding(one_lidar_batch_flat).to(device)

print('one_hsi_batch_embedded shape:', one_hsi_batch_embedded.shape)
print('one_lidar_batch_embedded shape:', one_lidar_batch_embedded.shape)


one_hsi_batch_embedded shape: torch.Size([1, 145, 128])
one_lidar_batch_embedded shape: torch.Size([1, 2, 128])


### Intialisation CrossAttention Class

In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch_6 = training_hsi_samples[:6]  # Shape: (6, 5, 5, 144)
lidar_batch_6 = training_lidar_samples[:6]  # Shape: (6, 5, 5, 1)

print("hsi_batch_6  shape:", hsi_batch_6 .shape)
print("lidar_batch_6 shape:", lidar_batch_6.shape)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
hsi_batch_6_flat = torch.from_numpy(hsi_batch_6.astype(np.float32).reshape(6, 25, 144, 1))
lidar_batch_6_flat = torch.from_numpy(lidar_batch_6.astype(np.float32).reshape(6, 25, 1, 1))

print("hsi_batch_6_flat  shape:", hsi_batch_6_flat .shape)
print("lidar_batch_6_flat shape:", lidar_batch_6_flat.shape)

hsi_batch_6_flat = hsi_batch_6_flat.to(device)  # Move the tensor to GPU
lidar_batch_6_flat = lidar_batch_6_flat.to(device)  # Move the tensor to GPU

hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
hsi_batch_6_embedded  = hsi_patch_embedding(hsi_batch_6_flat)
lidar_batch_6_embedded = lidar_patch_embedding(lidar_batch_6_flat)

device = torch.device("cuda:0")  # Define the device (GPU)

# Move the tensors to the desired device
hsi_batch_6_embedded = hsi_batch_6_embedded.to(device)
lidar_batch_6_embedded = lidar_batch_6_embedded.to(device)
print("hsi_batch_6_embedded  shape:", hsi_batch_6_embedded .shape)
print("lidar_batch_6_embedded shape:", lidar_batch_6_embedded.shape)

# Define the dimension of the model and the number of heads
d_model = hsi_config.emb_size  # the output dimension of PatchEmbedding
num_heads = hsi_config.heads  # the number of attention heads in the transformer

# Initialize CrossAttention module
cross_attention = CrossAttention(lidar_config, hsi_config).to(device)

# Apply the cross attention
output,attn_scores = cross_attention(lidar_batch_6_embedded, hsi_batch_6_embedded)


print("Cross attention output shape:", attn_scores.shape)
print("Output shape:", output.shape)




hsi_batch_6  shape: (6, 5, 5, 144)
lidar_batch_6 shape: (6, 5, 5, 1)
hsi_batch_6_flat  shape: torch.Size([6, 25, 144, 1])
lidar_batch_6_flat shape: torch.Size([6, 25, 1, 1])
hsi_batch_6_embedded  shape: torch.Size([6, 145, 128])
lidar_batch_6_embedded shape: torch.Size([6, 2, 128])
Cross attention output shape: torch.Size([6, 144, 144])
Output shape: torch.Size([6, 144, 512])


In [ ]:
cross_attention

CrossAttention(
  (to_q): Linear(in_features=128, out_features=512, bias=False)
  (to_k): Linear(in_features=128, out_features=512, bias=False)
  (to_v): Linear(in_features=128, out_features=512, bias=False)
  (to_out): Linear(in_features=512, out_features=144, bias=True)
)

In [ ]:
# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)
model

CrossAttentionModel(
  (hsi_patch_embedding): PatchEmbedding(
    (proj): Conv2d(25, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (lidar_patch_embedding): PatchEmbedding(
    (proj): Conv2d(25, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (cross_attention): CrossAttention(
    (to_q): Linear(in_features=128, out_features=512, bias=False)
    (to_k): Linear(in_features=128, out_features=512, bias=False)
    (to_v): Linear(in_features=128, out_features=512, bias=False)
    (to_out): Linear(in_features=512, out_features=144, bias=True)
  )
)

# 4.1  Training DataLoader for Cross Attention Module

In [ ]:
# 2.5 Training samples extraction
# The desired number of samples for each class
training_samples = [198, 190, 192, 188, 186, 182, 196, 191, 193, 191, 181, 192, 184, 181, 187]
training_samples_dict = {i+1: size for i, size in enumerate(training_samples)}

# Initialize a dictionary to store class count
class_count = {i: 0 for i in class_dict.keys()}

# Function to check if all classes have the required number of samples
def all_classes_completed(class_count, training_samples_dict):
    return all(class_count[class_num] == training_samples_dict[class_num] for class_num in class_dict.keys())

# Create an empty list to store patches and labels
hsi_samples = []
lidar_samples = []
labels = []

while not all_classes_completed(class_count, training_samples_dict):
    # Loop through the ground truth data
    for label in class_dict.keys():
        # Get the coordinates of the ground truth pixels
        coords = np.argwhere((gt_2013_data == label) & (mask > 0))

        # Shuffle the coordinates to randomize the patch extraction
        np.random.shuffle(coords)

        for coord in coords:
            i, j = coord
            # Calculate the patch indices
            i_start, i_end = i - patch_size // 2, i + patch_size // 2 + 1
            j_start, j_end = j - patch_size // 2, j + patch_size // 2 + 1

            # Check if the indices are within the bounds of the HSI data
            if i_start >= 0 and i_end <= hsi_2013_data.shape[0] and j_start >= 0 and j_end <= hsi_2013_data.shape[1]:
                # Extract the patch
                hsi_patch = hsi_2013_data[i_start:i_end, j_start:j_end, :]

                # Extract the LiDAR patch
                lidar_patch = lidar_2013_data[i_start:i_end, j_start:j_end, :]

                # If the class count is less than the required samples
                if class_count[label] < training_samples_dict[label]:
                    # Append the patch and its label to the list
                    hsi_samples.append(hsi_patch)
                    lidar_samples.append(lidar_patch)
                    labels.append(label)
                    class_count[label] += 1

                    # If all classes have the required number of samples, exit the loop
                    if all_classes_completed(class_count, training_samples_dict):
                        break

training_hsi_samples = np.array(hsi_samples)
training_lidar_samples = np.array(lidar_samples)
training_labels = np.array(labels)
print('training_hsi_samples shape:', training_hsi_samples.shape)
print('training_lidar_samples shape:', training_lidar_samples.shape)
print('training_labels shape:', training_labels.shape)

training_hsi_samples shape: (2832, 5, 5, 144)
training_lidar_samples shape: (2832, 5, 5, 1)
training_labels shape: (2832,)


In [ ]:
class HyperspectralDataset(Dataset):
    def __init__(self, hsi_samples, lidar_samples, labels):
        self.hsi_samples = hsi_samples
        self.lidar_samples = lidar_samples
        self.labels = labels

    def __len__(self):
        return len(self.hsi_samples)

    def __getitem__(self, idx):
        hsi_patch = self.hsi_samples[idx].float().to(device)
        lidar_patch = self.lidar_samples[idx].float().to(device)
        label = self.labels[idx]

        # Convert label to a tensor and reshape to match the model's output shape
        label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)

        return hsi_patch, lidar_patch, label


In [ ]:
labels_tensor = torch.tensor(training_labels)
labels_tensor -= 1  # Shift label values to the range [0, C-1]
print('labels_tensor:',labels_tensor.shape)

labels_tensor: torch.Size([2832])


In [ ]:
unique_classes = torch.unique(labels_tensor)
print("Unique classes:", unique_classes)

Unique classes: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


In [ ]:
from sklearn.model_selection import train_test_split

#Resahpe the data

# Move the input data to the desired device
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch = training_hsi_samples  # Shape: (2832, 5, 5, 144)
lidar_batch= training_lidar_samples # Shape: (2832, 5, 5, 1)
label_batch=training_labels
print("hsi_batch shape:", hsi_batch.shape)
print("lidar_batch shape:", lidar_batch.shape)
print("label_batch shape:", label_batch.shape)

## Reshape the data
hsi_batch_flat = torch.from_numpy(hsi_batch.astype(np.float32).reshape(2832, 25, 144, 1))
lidar_batch_flat = torch.from_numpy(lidar_batch.astype(np.float32).reshape(2832, 25, 1, 1))

print("hsi_batch shape:", hsi_batch_flat .shape)
print("lidar_batch shape:", lidar_batch_flat.shape)

hsi_batch_flat = hsi_batch_flat.to(device)  # Move the tensor to GPU
lidar_batch_flat = lidar_batch_flat.to(device)  # Move the tensor to GPU

# Split into train and test
hsi_samples_train, hsi_samples_test, lidar_samples_train, lidar_samples_test, labels_train, labels_test = train_test_split(
    hsi_batch_flat, lidar_batch_flat, label_batch, test_size=0.1, random_state=42)
print('labels_train shape:',labels_train.shape)

# Split train into train and validation
hsi_samples_train, hsi_samples_val, lidar_samples_train, lidar_samples_val, labels_train, labels_val = train_test_split(
    hsi_samples_train, lidar_samples_train, labels_train, test_size=0.60, random_state=42)
print('labels_train2 shape:',labels_train.shape)

# Now you have training, validation, and test data.
# Create Datasets
train_dataset = HyperspectralDataset(hsi_samples_train, lidar_samples_train, labels_train)
val_dataset = HyperspectralDataset(hsi_samples_val, lidar_samples_val, labels_val)
test_dataset = HyperspectralDataset(hsi_samples_test, lidar_samples_test, labels_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


hsi_batch shape: (2832, 5, 5, 144)
lidar_batch shape: (2832, 5, 5, 1)
label_batch shape: (2832,)
hsi_batch shape: torch.Size([2832, 25, 144, 1])
lidar_batch shape: torch.Size([2832, 25, 1, 1])
labels_train shape: (2548,)
labels_train2 shape: (1019,)


# 4.2 Training CorssAttentionMode ltraining

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import copy

# Define the batch size
batch_size = 32

# Create the data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.00) #loss 2.3866

# Initialize the best validation loss and best_model_wts before the training loop
best_val_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()

# Initialize the lowest_loss and best_model_wts before the training loop
lowest_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

num_epochs = 100

for epoch in range(num_epochs):
    for batch_idx, (hsi_batch, lidar_batch, label_batch) in enumerate(train_loader):
        # Move tensors to the configured device
        hsi_batch = hsi_batch.to(device)
        lidar_batch = lidar_batch.to(device)
        label_batch = label_batch.to(device)  # Reshape labels
        # print('hsi_batch shape:', hsi_batch.shape)
        # print('lidar_batch:', lidar_batch.shape)
        # print('label_batch shape:', label_batch.shape)

        # Forward pass
        output,attn_scores  = model(lidar_batch, hsi_batch)
        #print('output shape:', output.shape)

        # Calculate loss
        loss = criterion(output.transpose(1, 2), label_batch.squeeze(2))

        # Print loss every 10 batches
        #if (batch_idx + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


        # If the loss is lower than the current lowest, save the model's state
        if loss.item() < lowest_loss:
            lowest_loss = loss.item()
            best_model_wts = copy.deepcopy(model.state_dict())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Step the learning rate scheduler
        scheduler.step()

# Print finishing training
print('Finishing training')

# Save the best model weights
torch.save(best_model_wts, 'best_model_weights.pth')


Epoch [1/100], Batch [1/32], Loss: 13.3595
Epoch [1/100], Batch [2/32], Loss: 16.4099
Epoch [1/100], Batch [3/32], Loss: 15.5711
Epoch [1/100], Batch [4/32], Loss: 14.2605
Epoch [1/100], Batch [5/32], Loss: 14.4543
Epoch [1/100], Batch [6/32], Loss: 13.0786
Epoch [1/100], Batch [7/32], Loss: 13.1088
Epoch [1/100], Batch [8/32], Loss: 12.6681
Epoch [1/100], Batch [9/32], Loss: 11.5521
Epoch [1/100], Batch [10/32], Loss: 13.9038
Epoch [1/100], Batch [11/32], Loss: 11.8375
Epoch [1/100], Batch [12/32], Loss: 12.4341
Epoch [1/100], Batch [13/32], Loss: 10.0448
Epoch [1/100], Batch [14/32], Loss: 10.0942
Epoch [1/100], Batch [15/32], Loss: 11.1172
Epoch [1/100], Batch [16/32], Loss: 12.2169
Epoch [1/100], Batch [17/32], Loss: 10.8048
Epoch [1/100], Batch [18/32], Loss: 9.7726
Epoch [1/100], Batch [19/32], Loss: 10.0019
Epoch [1/100], Batch [20/32], Loss: 9.5897
Epoch [1/100], Batch [21/32], Loss: 8.2813
Epoch [1/100], Batch [22/32], Loss: 9.1455
Epoch [1/100], Batch [23/32], Loss: 11.0644
E

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import copy

# Define the batch size
batch_size = 32

# Create the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1)  # loss 2.6101
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min') # 2.5928
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0) # loss 2.6425
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=1) # loss 2.5945
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1) # loss 2..5951
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=1)#2.6039


# Initialize the best validation loss and best_model_wts before the training loop
best_val_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

num_epochs = 200
patience = 50  # Number of epochs with no improvement after which training will be stopped
no_improve_epochs = 0  # Number of epochs with no improvement

for epoch in range(num_epochs):
    # Training
    model.train()
    running_train_loss = 0.0
    for batch_idx, (hsi_batch, lidar_batch, label_batch) in enumerate(train_loader):
        # Move tensors to the configured device
        hsi_batch = hsi_batch.to(device)
        lidar_batch = lidar_batch.to(device)
        label_batch = label_batch.to(device)  # Reshape labels

        # Forward pass
        output, attn_scores  = model(lidar_batch, hsi_batch)

        # Calculate loss
        loss = criterion(output.transpose(1, 2), label_batch.squeeze(2))
        running_train_loss += loss.item() * hsi_batch.size(0)  # Multiply by batch size

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate epoch training loss
    epoch_train_loss = running_train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for hsi_batch, lidar_batch, label_batch in val_loader:
            # Move tensors to the configured device
            hsi_batch = hsi_batch.to(device)
            lidar_batch = lidar_batch.to(device)
            label_batch = label_batch.to(device)  # Reshape labels

            # Forward pass
            output = model(lidar_batch, hsi_batch)

            # Calculate loss
            loss = criterion(output[0].transpose(1, 2), label_batch.squeeze(2))
            running_val_loss += loss.item() * hsi_batch.size(0)  # Multiply by batch size

    # Calculate epoch validation loss
    epoch_val_loss = running_val_loss / len(val_loader.dataset)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')

    # If the validation loss is lower than the current best, save the model's state
    if epoch_val_loss < best_val_loss:
        print(f'Validation Loss Decreased({best_val_loss:.6f}--->{epoch_val_loss:.6f}) \t Saving The Model')
        best_val_loss = epoch_val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        no_improve_epochs = 0  # reset count
    else:
        no_improve_epochs += 1

    # Check early stopping condition
    if no_improve_epochs > patience:
        print('Early stopping!')
        model.load_state_dict(best_model_wts)  # load best model
        break

    # Step the learning rate scheduler
    #scheduler.step(epoch_val_loss)

print('Finished training')

# Save the best model weights
torch.save(best_model_wts, path+'best_model_weights.pth')


Epoch [1/200], Train Loss: 8.6001, Val Loss: 3.7772
Validation Loss Decreased(inf--->3.777221) 	 Saving The Model


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/200], Train Loss: 2.9858, Val Loss: 2.6757
Validation Loss Decreased(3.777221--->2.675684) 	 Saving The Model
Epoch [3/200], Train Loss: 2.6624, Val Loss: 2.6493
Validation Loss Decreased(2.675684--->2.649280) 	 Saving The Model
Epoch [4/200], Train Loss: 2.6368, Val Loss: 2.6352
Validation Loss Decreased(2.649280--->2.635213) 	 Saving The Model
Epoch [5/200], Train Loss: 2.6296, Val Loss: 2.6467
Epoch [6/200], Train Loss: 2.6185, Val Loss: 2.6095
Validation Loss Decreased(2.635213--->2.609454) 	 Saving The Model
Epoch [7/200], Train Loss: 2.6028, Val Loss: 2.6024
Validation Loss Decreased(2.609454--->2.602388) 	 Saving The Model
Epoch [8/200], Train Loss: 2.5959, Val Loss: 2.5985
Validation Loss Decreased(2.602388--->2.598497) 	 Saving The Model
Epoch [9/200], Train Loss: 2.5901, Val Loss: 2.5860
Validation Loss Decreased(2.598497--->2.586010) 	 Saving The Model
Epoch [10/200], Train Loss: 2.5848, Val Loss: 2.5921
Epoch [11/200], Train Loss: 2.5802, Val Loss: 2.5911
Epoch [12

In [ ]:
# Save the best model state
torch.save(best_model_wts, path+'best_model_weights.pth')


In [ ]:
# Load the saved model
best_model = CrossAttentionModel(hsi_config, lidar_config)

# Load the weights
best_model.load_state_dict(torch.load(path+'best_model_weights.pth'))

# Move the model to the GPU
best_model = best_model.to(device)

# Now your model is ready for making predictions on GPU


# 5.0 Load Trained Model

In [ ]:
# Load the model
model = CrossAttentionModel(hsi_config, lidar_config)
model.load_state_dict(torch.load(path+'best_model_weights.pth'))
model.eval()  # set the model to evaluation mode

# Move the model to the GPU
#
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
hsi_patch_embedding.pos_embedding 	 torch.Size([1, 145, 128])
hsi_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
hsi_patch_embedding.proj.weight 	 torch.Size([128, 25, 1, 1])
hsi_patch_embedding.proj.bias 	 torch.Size([128])
lidar_patch_embedding.pos_embedding 	 torch.Size([1, 2, 128])
lidar_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
lidar_patch_embedding.proj.weight 	 torch.Size([128, 25, 1, 1])
lidar_patch_embedding.proj.bias 	 torch.Size([128])
cross_attention.to_q.weight 	 torch.Size([512, 128])
cross_attention.to_k.weight 	 torch.Size([512, 128])
cross_attention.to_v.weight 	 torch.Size([512, 128])
cross_attention.to_out.weight 	 torch.Size([144, 512])
cross_attention.to_out.bias 	 torch.Size([144])


In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor])


In [ ]:
# Ensure the model is in evaluation mode
best_model.eval()

# Initialize a list to hold all the predictions and attention scores
all_predictions = []
all_attn_scores = []

# Loop over the validation set
for hsi_batch, lidar_batch, _ in val_loader:  # we don't need the labels for predictions
    # Move the batch to the desired device
    hsi_batch = hsi_batch.to(device)
    lidar_batch = lidar_batch.to(device)

    # Pass the batch through the model
    with torch.no_grad():
        output, attn_scores = best_model(lidar_batch, hsi_batch)

        # Add the predictions and attention scores to our lists
        all_predictions.append(output.cpu().numpy())
        all_attn_scores.append(attn_scores.cpu().numpy())

# Concatenate all predictions and attention scores into a single numpy array
all_predictions = np.concatenate(all_predictions)
all_attn_scores = np.concatenate(all_attn_scores)

# Now you can use the predictions and attention scores as needed
print('all_predictions shape:', all_predictions.shape )
print('all_attn_scores shape:', all_attn_scores.shape )


all_predictions shape: (1529, 144, 512)
all_attn_scores shape: (1529, 144, 144)


# Band Selection

Calculate the variance of the feature vectors for each band across all samples and then find the bands with the maximum variance

In [ ]:
# First, take the mean across all samples and heads. This will result in a single 144-dim vector.
mean_attention = np.mean(all_attn_scores, axis=(0, 1))
print('mean_attention:',mean_attention.shape)

# Then, sort the bands by attention. This will give you the band indices in descending order of attention.
sorted_band_indices = np.argsort(mean_attention)[::-1]
print('sorted_band_indices :',sorted_band_indices.shape)

# If you want to see the attention values as well, you can sort the mean_attention array in the same order.
sorted_attention_values = mean_attention[sorted_band_indices]
print('sorted_attention_values :',sorted_attention_values.shape)

N = 70  # Top N bands
top_N_bands = sorted_band_indices[:N]
top_N_attention_values = sorted_attention_values[:N]

# Print the top N band indices with their attention scores
print(f"Top {N} bands based on the mean attention score:")
for band_index, attention_value in zip(top_N_bands, top_N_attention_values):
    print(f"Band index: {band_index}, Mean attention score: {attention_value}")

# Print the list of top N band indices
print("\nList of top {} band indices:".format(N))
print(top_N_bands.tolist())


mean_attention: (144,)
sorted_band_indices : (144,)
sorted_attention_values : (144,)
Top 70 bands based on the mean attention score:
Band index: 115, Mean attention score: 32.019432067871094
Band index: 127, Mean attention score: 29.595396041870117
Band index: 63, Mean attention score: 21.864004135131836
Band index: 113, Mean attention score: 20.806163787841797
Band index: 54, Mean attention score: 20.27324867248535
Band index: 12, Mean attention score: 19.954082489013672
Band index: 60, Mean attention score: 18.420604705810547
Band index: 62, Mean attention score: 18.20339584350586
Band index: 4, Mean attention score: 17.818204879760742
Band index: 139, Mean attention score: 17.692548751831055
Band index: 50, Mean attention score: 17.607107162475586
Band index: 90, Mean attention score: 16.83207130432129
Band index: 51, Mean attention score: 15.624239921569824
Band index: 112, Mean attention score: 14.99621868133545
Band index: 75, Mean attention score: 14.399415016174316
Band index: 

In [ ]:
band_50=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101]
band_45=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21]
band_40=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56]
band_35=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58]
band_30=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85]
band_25=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81]
band_20=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48]
band_15=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75]
band_10=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139]
band_5=[115, 127, 63, 113, 54]
band_80=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65, 66, 45, 110, 93, 22, 16, 14, 123, 107, 100]
band_75=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65, 66, 45, 110, 93, 22]
band_70=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65]
band_65=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72]
band_60=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101]
band_55=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21]